In [1]:
import glob
import pandas as pd
import logging
from IPython.display import clear_output

In [2]:
logger = logging.getLogger(__name__)

logging.basicConfig(
    filename='logfile.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
from enum import IntEnum
from typing import List

class PktType(IntEnum):
    IPv4 = 0
    IPv6 = 1
    ICMP = 2
    IGMP = 3
    TCP_SYN = 4
    TCP_ACK = 5
    TCP_FIN = 6
    TCP_RST = 7
    UDP = 8
    UNKNOWN = 9

def set_pkt_type_code(cd: int, t: PktType) -> int:
    return cd | (1 << t)

def get_pkt_type_code(t: PktType) -> int:
    return 1 << t

def test_pkt_type_code(cd: int, t: PktType) -> bool:
    return cd & (1 << t) != 0

class StackType(IntEnum):
    F_ICMP = 0
    F_IGMP = 1
    F_TCP = 2
    F_UDP = 3
    F_UNKNOWN = 4


def get_pkt_stack_code(st: StackType) -> int:
    return 1 << st

def convert_packet2stack_code(pc: int) -> int:
    if test_pkt_type_code(pc, PktType.ICMP):
        return get_pkt_stack_code(StackType.F_ICMP)
    if test_pkt_type_code(pc, PktType.IGMP):
        return get_pkt_stack_code(StackType.F_IGMP)
    if test_pkt_type_code(pc, PktType.UDP):
        return get_pkt_stack_code(StackType.F_UDP)
    if test_pkt_type_code(pc, PktType.UNKNOWN):
        return get_pkt_stack_code(StackType.F_UNKNOWN)
    # Assuming the packet is TCP if none of the above conditions are met
    return get_pkt_stack_code(StackType.F_TCP)

def determine_class(labels):
    unique_labels = set(labels)  # Get the unique values in the labels list
    if unique_labels == {1}:
        return 1
    elif unique_labels == {0}:
        return 0
    else:
        return "N/A"

In [20]:
column_names = ["IP Version", "Source Address", "Destination Address", "Source Port", "Destination Port", "Time Stamp", "Packet Code", "Packet Length"]
data_type_format = ['int8', 'str', 'str', 'Int64', 'Int64', 'Int64', 'Int64', 'Int64'] 

dtype_dict = dict(zip(column_names, data_type_format))

n = 50

stats = []

for d in glob.glob("../data/*.data"):
    try:
        f_ = str(d).split('/')[-1].split('.')[0]
        print(f_)
        df = pd.read_csv(d, names=column_names, header=None, sep=" ", dtype=dtype_dict)
        
        with open(f"../data/{d.split('/')[-1].split('.')[0]}.label", 'r') as file:
            lines = file.readlines()
            if(len(lines) == 1):
                label_col = [c for c in lines[0]][:-1]
    
        if len(df) == len(label_col):
            df["Labels"] = label_col
        else:
            print("Length of the Label not equal to length of the data file")
    
        df['Labels'] = df['Labels'].astype(int)
    
        df = df[df['IP Version'] == 4 ]
        assert(len(df.dropna()) == len(df))
        
        df['Stack Code'] = df['Packet Code'].apply(convert_packet2stack_code)
        
        #assert(len(df) - len(df.drop_duplicates()) == 0)
    
        df = df.drop(columns=['Packet Code', 'IP Version'])
        
        #grouped_df = df.groupby(['Source Address', 'Destination Address', 'Source Port', 'Destination Port', 'Stack Code' ]).agg(Labels=('Labels', list), Packet_Count=('Labels', 'count')).reset_index()
        grouped_df = df.groupby(['Source Address', 'Destination Address']).agg(Labels=('Labels', list), Times=('Time Stamp', list), Pktl=('Packet Length', list), Packet_Count=('Labels', 'count')).reset_index()
        def all_equal(lst):
            return all(x == lst[0] for x in lst)
        #clear_output(wait=True)
        grouped_df['All_Equal'] = grouped_df['Labels'].apply(all_equal)        
        grouped_df = grouped_df[(grouped_df['All_Equal'] == True) & (grouped_df['Labels'].apply(lambda x: 1 in x))]

        grouped_df = grouped_df.drop(columns=['Labels', 'All_Equal'])
        
        print("Writing file:", len(grouped_df))
        grouped_df.to_csv(f'{f_}/atk_2tuple.csv', index=False)
    
    except Exception as e:
        
        logger.error(f"Error processing file {d}: {e}")
        logger.exception(e)
        logger.log(level=logging.INFO, msg=grouped_df['Class'].index)


spam50
Writing file: 111
webcompanion
Writing file: 202
telnetpwdla
Writing file: 200
ccleaner
Writing file: 91297
netbios_lrscan
Writing file: 517114
feiwo
Writing file: 1606
ntprdos
Writing file: 659
persistence
Writing file: 657
lrtcpdos10
Writing file: 77
scrapy
Writing file: 276
koler
Writing file: 418
agentinject
Writing file: 143734
magic
Writing file: 90110
dns_lrscan
Writing file: 502442
sshpwdla
Writing file: 400
sshpwdsm
Writing file: 60
trojanminer
Writing file: 202
plankton
Writing file: 750
sqlscan
Writing file: 77692
penetho
Writing file: 1006
ackport
Writing file: 26
lrtcpdos05
Writing file: 77
wannalocker
Writing file: 418
snojan
Writing file: 807
trickster
Writing file: 90110
zsone
Writing file: 1006
ransombo
Writing file: 418
httpscan
Writing file: 62972
dnsrdos
Writing file: 209
dnsscan
Writing file: 63174
spam100
Writing file: 191
snmp_lrscan
Writing file: 195426
httpsscan
Writing file: 126640
telnetpwdmd
Writing file: 100
telnet_lrscan
Writing file: 1235684
http_l